In [8]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt
import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime

In [10]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [11]:
# calling API 
data=YahooFinancials('aapl').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
aapl_df = pd.DataFrame(data['AAPL']['prices'])
aapl_df =aapl_df.drop(['high','low','open','volume','date','adjclose'], axis=1)
aapl_df.rename(columns={'formatted_date':'date'},inplace=True)
aapl_df['date']=pd.to_datetime(aapl_df['date'])

In [12]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [13]:
aapl_df

,close,date
0,64.857498,2019-11-07
1,65.035004,2019-11-08
2,65.550003,2019-11-11
3,65.489998,2019-11-12
4,66.117500,2019-11-13
...,...,...
754,138.380005,2022-11-04
755,138.919998,2022-11-07
756,139.500000,2022-11-08
757,134.869995,2022-11-09


In [14]:
# Scale data for ML engine
scaler = MinMaxScaler()
aapl_df['scaled_close'] = scaler.fit_transform(np.expand_dims(aapl_df['close'].values, axis=1))

In [15]:
def PrepareData(days):
    df = aapl_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [16]:
PrepareData(3) # 3 days


(          close       date  scaled_close    future
 0     64.857498 2019-11-07      0.069609  0.074632
 1     65.035004 2019-11-08      0.071019  0.079616
 2     65.550003 2019-11-11      0.075109  0.075982
 3     65.489998 2019-11-12      0.074632  0.082177
 4     66.117500 2019-11-13      0.079616  0.084837
 ..          ...        ...           ...       ...
 751  150.649994 2022-11-01      0.750948  0.653503
 752  145.029999 2022-11-02      0.706316  0.657792
 753  138.880005 2022-11-03      0.657474  0.662398
 754  138.380005 2022-11-04      0.653503  0.625628
 755  138.919998 2022-11-07      0.657792  0.720928
 
 [756 rows x 4 columns], array([[0.7913714 ],
        [0.7723113 ],
        [0.750948  ],
        [0.7063157 ],
        [0.6574742 ],
        [0.65350336],
        [0.65779185],
        [0.66239804],
        [0.6256279 ],
        [0.7209284 ]], dtype=float32), array([[[0.06960906694502711, Timestamp('2019-11-07 00:00:00')],
         [0.07101876376387228, Timestamp('2019-1

In [17]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [18]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
94/94 [==============================] - 5s 14ms/step - loss: 0.0233
Epoch 2/80
94/94 [==============================] - 2s 20ms/step - loss: 0.0049
Epoch 3/80
94/94 [==============================] - 2s 21ms/step - loss: 0.0041
Epoch 4/80
94/94 [==============================] - 2s 21ms/step - loss: 0.0037
Epoch 5/80
94/94 [==============================] - 2s 21ms/step - loss: 0.0038
Epoch 6/80
94/94 [==============================] - 2s 20ms/step - loss: 0.0032
Epoch 7/80
94/94 [==============================] - 2s 20ms/step - loss: 0.0029
Epoch 8/80
94/94 [==============================] - 2s 21ms/step - loss: 0.0027
Epoch 9/80
94/94 [==============================] - 2s 23ms/step - loss: 0.0027
Epoch 10/80
94/94 [==============================] - 2s 21ms/step - loss: 0.0024
Epoch 11/80
94/94 [==============================] - 2s 21ms/step - loss: 0.0024
Epoch 12/80
94/94 [==============================] - 2s 22ms/step - loss: 0.0023
Epoch 13/80
94/94 [==================

In [19]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (143.16$, 142.62$, 142.76$)


In [21]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
aapl_lstm=pd.DataFrame()
aapl_lstm['future days']=days
aapl_lstm['price']=predictions
aapl_lstm['ticker']='AAPL'

aapl_lstm.to_csv('aapl_lstm.csv') 
files.download('aapl_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>